In [3]:
!pip install keras tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 4.0 MB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 3.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 5.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 4.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 4.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 6.1 MB/s 

In [4]:
import os
import re
import json
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

import pyarrow
import time
from pyod.models.ocsvm import OCSVM
from pyod.models import auto_encoder 


In [5]:
os.chdir("./")
print("Current directory:", os.getcwd())

import pandas as pd
import glob
import os
# Find all matching files
files = sorted(glob.glob("sequence_hdfs_*.bin"), key=os.path.getmtime)
if not files:
    raise FileNotFoundError("No sequence_hdfs_*.bin files found!")
# Pick the most recent one
latest_file = files[-1]
print(f"Loading: {latest_file}")
# Load as Feather
df_sequence = pd.read_feather(latest_file)
print(df_sequence.columns)
print(df_sequence.shape)
print(df_sequence.head(5))

Current directory: /home/andy/6309GP
Loading: sequence_hdfs_20251024_183457.bin
Index(['SessionId', 'EventSequence', 'Label'], dtype='object')
(575061, 3)
                  SessionId  \
0  blk_-1608999687919862906   
1   blk_7503483334202473044   
2  blk_-3544583377289625738   
3  blk_-9073992586687739851   
4   blk_7854771516489510256   

                                       EventSequence  Label  
0  E5 E22 E5 E5 E11 E11 E9 E9 E11 E9 E26 E26 E26 ...      0  
1  E5 E5 E22 E5 E11 E9 E11 E9 E11 E9 E26 E26 E26 ...      0  
2  E5 E22 E5 E5 E11 E9 E11 E9 E11 E9 E3 E26 E26 E...      1  
3  E5 E22 E5 E5 E11 E9 E11 E9 E11 E9 E26 E26 E26 ...      0  
4  E5 E5 E22 E5 E11 E9 E11 E9 E11 E9 E26 E26 E26 ...      0  


In [6]:
from sklearn.model_selection import train_test_split

def split_sequence_dataset(df, test_size=0.3, random_state=42):
    """
    Split a sequence dataset into training and evaluation sets.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with columns ['SessionId', 'EventSequence', 'Label'].
    test_size : float
        Fraction of the dataset to reserve for evaluation (default 0.3).
    random_state : int
        Random seed for reproducibility.

    Returns
    -------
    sequence_train : pd.DataFrame
    sequence_eval : pd.DataFrame
    """
    sequence_train, sequence_eval = train_test_split(
        df,
        test_size=test_size,
        random_state=random_state,
        stratify=df["Label"],  # preserve label distribution
        shuffle=True
    )
    return sequence_train.reset_index(drop=True), sequence_eval.reset_index(drop=True)

sequence_train, sequence_eval = split_sequence_dataset(df_sequence)
print(sequence_train.shape, sequence_eval.shape)
print(sequence_train["Label"].value_counts(normalize=True))
print(sequence_eval["Label"].value_counts(normalize=True))

(402542, 3) (172519, 3)
Label
0    0.970719
1    0.029281
Name: proportion, dtype: float64
Label
0    0.970722
1    0.029278
Name: proportion, dtype: float64


In [7]:
def get_normal_subset(df_train, label_col="Label"):
    return df_train[df_train[label_col] == 0].reset_index(drop=True)

sequence_train_normal = get_normal_subset(sequence_train)
print(sequence_train_normal.shape)
print(sequence_train_normal["Label"].value_counts())

(390755, 3)
Label
0    390755
Name: count, dtype: int64


In [8]:
def identity_tokenizer(s: str):
    # For pre-tokenized input (space-separated)
    return s.split()

tfidf_id = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False, preprocessor=None)
tfidf_matrix_id = tfidf_id.fit_transform(df_sequence["EventSequence"].astype(str))

feature_names_id = np.array(tfidf_id.get_feature_names_out())
X = tfidf_id.fit_transform(df_sequence["EventSequence"])
y = df_sequence["Label"].values

X_train, X_eval, y_train, y_eval = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

mask_normal = (y_train == 0)
X_train_normal = X_train[mask_normal]
print("Train normal:", X_train_normal.shape, "Eval:", X_eval.shape)


/home/andy/andyVenv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Train normal: (390755, 29) Eval: (172519, 29)


threat to validity: the use of autoencoder expects a matrix with the same size. This is a poor combination when applied with MCV and TF-IDF: 

IF there are novel templates in eval set in comparison to training set, the autoencoder breaks. 

In post, it would be better to use an embedding such as word to vect where the matrix is of a fixed dimension. 

In [34]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

def evaluate_autoencoder_results(result_df):
    """
    Compute classification metrics from AutoEncoder results.

    Parameters
    ----------
    result_df : pd.DataFrame
        Must contain columns: ['true_label', 'pred_label', 'reconstruction_error']

    Returns
    -------
    metrics : dict
        Dictionary of precision, recall, f1, and auc scores.
    """

    y_true = result_df["true_label"].astype(int)
    y_pred = result_df["pred_label"].astype(int)
    y_score = result_df["reconstruction_error"].values

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # ROC AUC can be computed on the reconstruction error scores
    auc = roc_auc_score(y_true, y_score)

    metrics = {
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "auc": auc
    }

    print("\n📊 Evaluation metrics:")
    for k, v in metrics.items():
        print(f"{k:10s}: {v:.4f}")

    return metrics

In [33]:
import pandas as pd
import numpy as np
from pyod.models.auto_encoder import AutoEncoder

def train_and_evaluate_autoencoder(X_train, X_eval, y_train, y_eval,
                                   contamination=0.001, hidden_neurons=[64, 32],
                                   epochs=20, batch_size=32, lr=1e-3):
    """
    Train a PyOD AutoEncoder on X_train (normal samples) and evaluate on X_eval.

    Parameters
    ----------
    X_train : array-like or sparse matrix
        Training feature matrix (only normal samples).
    X_eval : array-like or sparse matrix
        Evaluation feature matrix.
    y_train, y_eval : array-like
        Labels for training and evaluation sets.
    contamination : float
        Expected proportion of anomalies in the data.
    hidden_neurons : list
        Hidden layer sizes for encoder/decoder symmetry.
    epochs : int
        Number of training epochs.
    batch_size : int
        Training batch size.
    lr : float
        Learning rate.

    Returns
    -------
    result_df : pd.DataFrame
        DataFrame with columns:
        ['true_label', 'pred_label', 'reconstruction_error']
    model : AutoEncoder
        Trained PyOD autoencoder model.
    """

    print(f"Training AutoEncoder on {X_train.shape[0]} samples, "
          f"evaluating on {X_eval.shape[0]} samples...")

    # Initialize and fit AutoEncoder
    model = AutoEncoder(hidden_neuron_list=hidden_neurons,
                        contamination=contamination,
                        epoch_num=epochs,
                        batch_size=batch_size,
                        lr=lr,
                        verbose=1)

    model.fit(X_train.toarray())

    # Predict on evaluation set
    y_pred = model.predict(X_eval.toarray())                     # binary labels (0=inlier, 1=outlier)
    scores = model.decision_function(X_eval.toarray())           # reconstruction errors

    # Build output dataframe
    result_df = pd.DataFrame({
        "true_label": y_eval,
        "pred_label": y_pred,
        "reconstruction_error": scores
    })

    print("\n✅ Training complete.")
    print(result_df["pred_label"].value_counts())
    print("Mean reconstruction error:", np.mean(scores))

    return result_df, model



In [31]:
import pandas as pd
import numpy as np

def analyze_reconstruction_error_by_label(result_df):
    """
    Compute average and standard deviation of reconstruction errors
    for normal (label=0) and anomaly (label=1) samples.

    Parameters
    ----------
    result_df : pd.DataFrame
        DataFrame with at least columns ['true_label', 'reconstruction_error'].

    Returns
    -------
    summary_df : pd.DataFrame
        Summary of mean and std reconstruction error by label.
    """

    if not {"true_label", "reconstruction_error"}.issubset(result_df.columns):
        raise ValueError("result_df must contain 'true_label' and 'reconstruction_error' columns")

    summary = (
        result_df.groupby("true_label")["reconstruction_error"]
        .agg(["mean", "std", "min", "max", "count"])
        .rename(index={0: "Normal (0)", 1: "Anomaly (1)"})
    )

    print("\n📊 Reconstruction Error Summary by Label:")
    print(summary)

    diff_ratio = summary.loc["Anomaly (1)", "mean"] / summary.loc["Normal (0)", "mean"]
    print(f"\nMean error ratio (Anomaly / Normal): {diff_ratio:.2f}")

    if diff_ratio < 1.2:
        print("⚠️  Weak separation — model may be underfitting or over-regularized.")
    elif diff_ratio > 2.0:
        print("✅  Strong separation — anomalies reconstruct significantly worse.")
    else:
        print("ℹ️  Moderate separation — may improve with tuning (epochs, hidden layers, or dropout).")

    return summary

In [ ]:
result_df, ae_model = train_and_evaluate_autoencoder(
    X_train_normal, X_eval, y_train=None, y_eval=y_eval,
    contamination=0.1, hidden_neurons=[128, 64], epochs=30
)

print(result_df.head(10))
metrics = evaluate_autoencoder_results(result_df)

result_df, ae_model = train_and_evaluate_autoencoder(
    X_train_normal, X_eval, y_train=None, y_eval=y_eval,
    contamination=0.01, hidden_neurons=[128, 64], epochs=30
)

print(result_df.head(10))
metrics = evaluate_autoencoder_results(result_df)

result_df, ae_model = train_and_evaluate_autoencoder(
    X_train_normal, X_eval, y_train=None, y_eval=y_eval,
    contamination=0.005, hidden_neurons=[128, 64], epochs=30
)

print(result_df.head(10))
metrics = evaluate_autoencoder_results(result_df)

result_df, ae_model = train_and_evaluate_autoencoder(
    X_train_normal, X_eval, y_train=None, y_eval=y_eval,
    contamination=0.001, hidden_neurons=[128, 64], epochs=30
)

print(result_df.head(10))
metrics = evaluate_autoencoder_results(result_df)


Training AutoEncoder on 390755 samples, evaluating on 172519 samples...


Training: 100%|██████████| 30/30 [06:29<00:00, 12.98s/it]



✅ Training complete.
pred_label
0    151367
1     21152
Name: count, dtype: int64
Mean reconstruction error: 606559.75
   true_label  pred_label  reconstruction_error
0           0           0              0.054559
1           0           0              0.054559
2           0           0              0.054559
3           0           1             38.753941
4           0           0              0.054559
5           0           0              0.380731
6           0           0              0.054559
7           0           0              0.054559
8           0           1              1.399192
9           0           1              0.796620

📊 Evaluation metrics:
precision : 0.2388
recall    : 1.0000
f1_score  : 0.3855
auc       : 0.9989
Training AutoEncoder on 390755 samples, evaluating on 172519 samples...


Training: 100%|██████████| 30/30 [06:27<00:00, 12.93s/it]



✅ Training complete.
pred_label
0    165871
1      6648
Name: count, dtype: int64
Mean reconstruction error: 606559.75
   true_label  pred_label  reconstruction_error
0           0           0              0.054559
1           0           0              0.054559
2           0           0              0.054559
3           0           1             38.753941
4           0           0              0.054559
5           0           0              0.380731
6           0           0              0.054559
7           0           0              0.054559
8           0           1              1.399192
9           0           0              0.796620

📊 Evaluation metrics:
precision : 0.7598
recall    : 1.0000
f1_score  : 0.8635
auc       : 0.9989
Training AutoEncoder on 390755 samples, evaluating on 172519 samples...


Training: 100%|██████████| 30/30 [06:34<00:00, 13.15s/it]



✅ Training complete.
pred_label
0    166721
1      5798
Name: count, dtype: int64
Mean reconstruction error: 606559.75
   true_label  pred_label  reconstruction_error
0           0           0              0.054559
1           0           0              0.054559
2           0           0              0.054559
3           0           1             38.753941
4           0           0              0.054559
5           0           0              0.380731
6           0           0              0.054559
7           0           0              0.054559
8           0           0              1.399192
9           0           0              0.796620

📊 Evaluation metrics:
precision : 0.8712
recall    : 1.0000
f1_score  : 0.9311
auc       : 0.9989
Training AutoEncoder on 390755 samples, evaluating on 172519 samples...


Training: 100%|██████████| 30/30 [06:38<00:00, 13.30s/it]



✅ Training complete.
pred_label
0    169034
1      3485
Name: count, dtype: int64
Mean reconstruction error: 606559.75
   true_label  pred_label  reconstruction_error
0           0           0              0.054559
1           0           0              0.054559
2           0           0              0.054559
3           0           0             38.753941
4           0           0              0.054559
5           0           0              0.380731
6           0           0              0.054559
7           0           0              0.054559
8           0           0              1.399192
9           0           0              0.796620

📊 Evaluation metrics:
precision : 0.9529
recall    : 0.6575
f1_score  : 0.7781
auc       : 0.9989


In [35]:
result_df, ae_model = train_and_evaluate_autoencoder(
    X_train_normal, X_eval, y_train=None, y_eval=y_eval,
    contamination=0.01, hidden_neurons=[128, 64], epochs=30
)

print(result_df.head(10))
metrics = evaluate_autoencoder_results(result_df)
analyze_reconstruction_error_by_label(result_df)

result_df, ae_model = train_and_evaluate_autoencoder(
    X_train_normal, X_eval, y_train=None, y_eval=y_eval,
    contamination=0.005, hidden_neurons=[128, 64], epochs=30
)

print(result_df.head(10))
metrics = evaluate_autoencoder_results(result_df)
analyze_reconstruction_error_by_label(result_df)

result_df, ae_model = train_and_evaluate_autoencoder(
    X_train_normal, X_eval, y_train=None, y_eval=y_eval,
    contamination=0.001, hidden_neurons=[128, 64], epochs=30
)

print(result_df.head(10))
metrics = evaluate_autoencoder_results(result_df)
analyze_reconstruction_error_by_label(result_df)

Training AutoEncoder on 390755 samples, evaluating on 172519 samples...


Training: 100%|██████████| 30/30 [06:21<00:00, 12.72s/it]



✅ Training complete.
pred_label
0    165871
1      6648
Name: count, dtype: int64
Mean reconstruction error: 606559.75
   true_label  pred_label  reconstruction_error
0           0           0              0.054559
1           0           0              0.054559
2           0           0              0.054559
3           0           1             38.753941
4           0           0              0.054559
5           0           0              0.380731
6           0           0              0.054559
7           0           0              0.054559
8           0           1              1.399192
9           0           0              0.796620

📊 Evaluation metrics:
precision : 0.7598
recall    : 1.0000
f1_score  : 0.8635
auc       : 0.9989

📊 Reconstruction Error Summary by Label:
                     mean           std       min           max   count
true_label                                                             
Normal (0)   4.107166e-01  2.283976e+00  0.054559  1.272052e+02  16

Training: 100%|██████████| 30/30 [06:19<00:00, 12.63s/it]



✅ Training complete.
pred_label
0    166721
1      5798
Name: count, dtype: int64
Mean reconstruction error: 606559.75
   true_label  pred_label  reconstruction_error
0           0           0              0.054559
1           0           0              0.054559
2           0           0              0.054559
3           0           1             38.753941
4           0           0              0.054559
5           0           0              0.380731
6           0           0              0.054559
7           0           0              0.054559
8           0           0              1.399192
9           0           0              0.796620

📊 Evaluation metrics:
precision : 0.8712
recall    : 1.0000
f1_score  : 0.9311
auc       : 0.9989

📊 Reconstruction Error Summary by Label:
                     mean           std       min           max   count
true_label                                                             
Normal (0)   4.107166e-01  2.283976e+00  0.054559  1.272052e+02  16

Training:  50%|█████     | 15/30 [03:18<03:18, 13.25s/it]


KeyboardInterrupt: 

In [36]:
result_df, ae_model = train_and_evaluate_autoencoder(
    X_train_normal, X_eval, y_train=None, y_eval=y_eval,
    contamination=0.002, hidden_neurons=[128, 64], epochs=30
)

print(result_df.head(10))
metrics = evaluate_autoencoder_results(result_df)
analyze_reconstruction_error_by_label(result_df)

result_df, ae_model = train_and_evaluate_autoencoder(
    X_train_normal, X_eval, y_train=None, y_eval=y_eval,
    contamination=0.003, hidden_neurons=[128, 64], epochs=30
)

print(result_df.head(10))
metrics = evaluate_autoencoder_results(result_df)
analyze_reconstruction_error_by_label(result_df)

result_df, ae_model = train_and_evaluate_autoencoder(
    X_train_normal, X_eval, y_train=None, y_eval=y_eval,
    contamination=0.004, hidden_neurons=[128, 64], epochs=30
)

print(result_df.head(10))
metrics = evaluate_autoencoder_results(result_df)
analyze_reconstruction_error_by_label(result_df)

Training AutoEncoder on 390755 samples, evaluating on 172519 samples...


Training: 100%|██████████| 30/30 [06:30<00:00, 13.01s/it]



✅ Training complete.
pred_label
0    169034
1      3485
Name: count, dtype: int64
Mean reconstruction error: 606559.75
   true_label  pred_label  reconstruction_error
0           0           0              0.054559
1           0           0              0.054559
2           0           0              0.054559
3           0           0             38.753941
4           0           0              0.054559
5           0           0              0.380731
6           0           0              0.054559
7           0           0              0.054559
8           0           0              1.399192
9           0           0              0.796620

📊 Evaluation metrics:
precision : 0.9529
recall    : 0.6575
f1_score  : 0.7781
auc       : 0.9989

📊 Reconstruction Error Summary by Label:
                     mean           std       min           max   count
true_label                                                             
Normal (0)   4.107166e-01  2.283976e+00  0.054559  1.272052e+02  16

Training: 100%|██████████| 30/30 [06:40<00:00, 13.35s/it]



✅ Training complete.
pred_label
0    168392
1      4127
Name: count, dtype: int64
Mean reconstruction error: 606559.75
   true_label  pred_label  reconstruction_error
0           0           0              0.054559
1           0           0              0.054559
2           0           0              0.054559
3           0           1             38.753941
4           0           0              0.054559
5           0           0              0.380731
6           0           0              0.054559
7           0           0              0.054559
8           0           0              1.399192
9           0           0              0.796620

📊 Evaluation metrics:
precision : 0.8885
recall    : 0.7260
f1_score  : 0.7991
auc       : 0.9989

📊 Reconstruction Error Summary by Label:
                     mean           std       min           max   count
true_label                                                             
Normal (0)   4.107166e-01  2.283976e+00  0.054559  1.272052e+02  16

Training: 100%|██████████| 30/30 [06:40<00:00, 13.34s/it]



✅ Training complete.
pred_label
0    166812
1      5707
Name: count, dtype: int64
Mean reconstruction error: 606559.75
   true_label  pred_label  reconstruction_error
0           0           0              0.054559
1           0           0              0.054559
2           0           0              0.054559
3           0           1             38.753941
4           0           0              0.054559
5           0           0              0.380731
6           0           0              0.054559
7           0           0              0.054559
8           0           0              1.399192
9           0           0              0.796620

📊 Evaluation metrics:
precision : 0.8851
recall    : 1.0000
f1_score  : 0.9390
auc       : 0.9989

📊 Reconstruction Error Summary by Label:
                     mean           std       min           max   count
true_label                                                             
Normal (0)   4.107166e-01  2.283976e+00  0.054559  1.272052e+02  16

,mean,std,min,max,count
true_label,,,,,
Normal (0),4.107166e-01,2.283976e+00,0.054559,1.272052e+02,167468
Anomaly (1),2.071729e+07,3.892227e+07,3.614228,9.946272e+07,5051


In [41]:
for i in range (0,20):
    result_df, ae_model = train_and_evaluate_autoencoder(
    X_train_normal, X_eval, y_train=None, y_eval=y_eval,
    contamination=0.001 * (i+1), hidden_neurons=[128, 64], epochs=30
    )

    print(result_df.head(10))
    metrics = evaluate_autoencoder_results(result_df)

Training AutoEncoder on 390755 samples, evaluating on 172519 samples...


Training: 100%|██████████| 30/30 [06:54<00:00, 13.82s/it]



✅ Training complete.
pred_label
0    169034
1      3485
Name: count, dtype: int64
Mean reconstruction error: 606559.75
   true_label  pred_label  reconstruction_error
0           0           0              0.054559
1           0           0              0.054559
2           0           0              0.054559
3           0           0             38.753941
4           0           0              0.054559
5           0           0              0.380731
6           0           0              0.054559
7           0           0              0.054559
8           0           0              1.399192
9           0           0              0.796620

📊 Evaluation metrics:
precision : 0.9529
recall    : 0.6575
f1_score  : 0.7781
auc       : 0.9989
Training AutoEncoder on 390755 samples, evaluating on 172519 samples...


Training: 100%|██████████| 30/30 [06:34<00:00, 13.16s/it]



✅ Training complete.
pred_label
0    169034
1      3485
Name: count, dtype: int64
Mean reconstruction error: 606559.75
   true_label  pred_label  reconstruction_error
0           0           0              0.054559
1           0           0              0.054559
2           0           0              0.054559
3           0           0             38.753941
4           0           0              0.054559
5           0           0              0.380731
6           0           0              0.054559
7           0           0              0.054559
8           0           0              1.399192
9           0           0              0.796620

📊 Evaluation metrics:
precision : 0.9529
recall    : 0.6575
f1_score  : 0.7781
auc       : 0.9989
Training AutoEncoder on 390755 samples, evaluating on 172519 samples...


Training:  10%|█         | 3/30 [00:46<07:01, 15.61s/it]


KeyboardInterrupt: 

In [37]:
metrics = evaluate_autoencoder_results(result_df)


📊 Evaluation metrics:
precision : 0.9529
recall    : 0.6575
f1_score  : 0.7781
auc       : 0.9989


In [3]:
model = OCSVM(kernel='rbf', gamma='auto', contamination=0.1)


In [ ]:
#pyod example
from pyod.models import auto_encoder
from pyod.utils.data import generate_data
auto_encoder.AutoEncoder()

# Generate synthetic data
X_train, X_test, y_train, y_test = generate_data(n_train=500, n_test=100, n_features=20)

# Initialize AutoEncoder
clf = auto_encoder.AutoEncoder(
    hidden_neuron_list=[64, 32],
    epoch_num=10,
    batch_size=32,
    device='cuda',   # or 'cpu' if no GPU
    contamination=0.1
)

# Fit model
clf.fit(X_train)

# Predict anomalies
y_pred = clf.predict(X_test)

# Print sample results
print("Sample anomaly predictions:", y_pred[:10])
print("Sample reconstruction errors:", clf.decision_scores_[:10])

Training: 100%|██████████| 10/10 [00:00<00:00, 19.54it/s]


Sample anomaly predictions: [0 0 0 0 0 0 0 0 0 0]
Sample reconstruction errors: [1.7826098 1.1180751 1.8728502 1.4407952 1.6203517 1.7224503 1.6781187
 1.2661917 1.4927775 2.1730125]


In [42]:
import pandas as pd
import numpy as np
from pyod.models.ocsvm import OCSVM

def train_and_evaluate_ocsvm(X_train, X_eval, y_train, y_eval,
                             kernel='rbf', nu=0.05, gamma='auto', contamination=0.05):
    """
    Train a PyOD One-Class SVM (OCSVM) on X_train (normal samples) and evaluate on X_eval.

    Parameters
    ----------
    X_train : array-like or sparse matrix
        Training feature matrix (only normal samples).
    X_eval : array-like or sparse matrix
        Evaluation feature matrix.
    y_train, y_eval : array-like
        Labels for training and evaluation sets.
    kernel : str
        Kernel type ('linear', 'poly', 'rbf', 'sigmoid').
    nu : float
        Expected fraction of outliers in training data (upper bound).
    gamma : str or float
        Kernel coefficient ('scale', 'auto', or numeric value).
    contamination : float
        Expected fraction of anomalies in the data (used for thresholding).

    Returns
    -------
    result_df : pd.DataFrame
        DataFrame with columns:
        ['true_label', 'pred_label', 'anomaly_score']
    model : OCSVM
        Trained PyOD OCSVM model.
    """

    print(f"Training OCSVM on {X_train.shape[0]} normal samples, "
          f"evaluating on {X_eval.shape[0]} samples...")

    # Initialize and train the model
    model = OCSVM(kernel=kernel, nu=nu, gamma=gamma, contamination=contamination)
    model.fit(X_train.toarray())

    # Predict on evaluation set
    y_pred = model.predict(X_eval.toarray())              # 0=inlier, 1=outlier
    scores = model.decision_function(X_eval.toarray())    # anomaly scores (higher = more normal)

    # Build result DataFrame
    result_df = pd.DataFrame({
        "true_label": y_eval,
        "pred_label": y_pred,
        "anomaly_score": scores
    })

    # Display summary
    print("\n✅ Training complete.")
    print("Predicted label counts:")
    print(result_df["pred_label"].value_counts())
    print("Mean anomaly score:", np.mean(scores))

    return result_df, model

In [ ]:
# def identity_tokenizer(s: str):
#     # For pre-tokenized input (space-separated)
#     return s.split()

# tfidf_id = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False, preprocessor=None)
# tfidf_matrix_id = tfidf_id.fit_transform(df_sequence["EventSequence"].astype(str))

# feature_names_id = np.array(tfidf_id.get_feature_names_out())
# X = tfidf_id.fit_transform(df_sequence["EventSequence"])
# y = df_sequence["Label"].values

# X_train, X_eval, y_train, y_eval = train_test_split(
#     X, y, test_size=0.3, random_state=42, stratify=y
# )

# mask_normal = (y_train == 0)
# X_train_normal = X_train[mask_normal]
# print("Train normal:", X_train_normal.shape, "Eval:", X_eval.shape)

# Train and evaluate
result_df, ocsvm_model = train_and_evaluate_ocsvm(
    X_train=X_train_normal,
    X_eval=X_eval,
    y_train=y_train,
    y_eval=y_eval,
    kernel='rbf',
    nu=0.05,
    gamma='scale',
    contamination=0.05
)

# Optional: evaluate performance metrics
from sklearn.metrics import classification_report, roc_auc_score

print(classification_report(result_df["true_label"], result_df["pred_label"]))
print("ROC AUC:", roc_auc_score(result_df["true_label"], result_df["anomaly_score"]))

Training OCSVM on 390755 normal samples, evaluating on 172519 samples...


Takes too long. may i try smth else

In [20]:
import pandas as pd
import numpy as np
from pyod.models.iforest import IForest

def train_and_evaluate_iforest(X_train, X_eval, y_train, y_eval,
                               contamination=0.05, n_estimators=100,
                               max_samples='auto', n_jobs=-1, random_state=42):
    print(f"training IForest contamination={contamination}, n_estimators={n_estimators}, max_samples={max_samples}, n_jobs={n_jobs}, random_state={random_state}")
    """
    Train a PyOD Isolation Forest on X_train (normal samples) and evaluate on X_eval.

    Parameters
    ----------
    X_train : array-like or sparse matrix
        Training feature matrix (only normal samples).
    X_eval : array-like or sparse matrix
        Evaluation feature matrix.
    y_train, y_eval : array-like
        Labels for training and evaluation sets.
    contamination : float
        Expected proportion of anomalies in the data.
    n_estimators : int
        Number of base estimators (trees).
    max_samples : int or 'auto'
        Number of samples to draw to train each base estimator.
    n_jobs : int
        Number of parallel threads to use (-1 = all cores).
    random_state : int
        Random seed for reproducibility.

    Returns
    -------
    result_df : pd.DataFrame
        DataFrame with columns:
        ['true_label', 'pred_label', 'anomaly_score']
    model : IForest
        Trained PyOD Isolation Forest model.
    """

    print(f"Training IsolationForest on {X_train.shape[0]} normal samples, "
          f"evaluating on {X_eval.shape[0]} samples...")

    # Initialize model
    model = IForest(contamination=contamination,
                    n_estimators=n_estimators,
                    max_samples=max_samples,
                    n_jobs=n_jobs,
                    random_state=random_state)

    # Fit model on training data
    model.fit(X_train.toarray())

    # Predict on evaluation set
    y_pred = model.predict(X_eval.toarray())               # 0=inlier, 1=outlier
    scores = model.decision_function(X_eval.toarray())     # anomaly scores (higher = more normal)

    # Build output dataframe
    result_df = pd.DataFrame({
        "true_label": y_eval,
        "pred_label": y_pred,
        "anomaly_score": scores
    })

    print("\n✅ Training complete.")
    print("Predicted label counts:")
    print(result_df["pred_label"].value_counts())
    print("Mean anomaly score:", np.mean(scores))

    return result_df, model

In [19]:
result_df, iforest_model = train_and_evaluate_iforest(
    X_train=X_train_normal,
    X_eval=X_eval,
    y_train=y_train,
    y_eval=y_eval,
    contamination=0.05,
    n_estimators=10000,
    n_jobs=-1  # ✅ use all cores
)

# Evaluate performance
from sklearn.metrics import classification_report, roc_auc_score
print(classification_report(result_df["true_label"], result_df["pred_label"]))

Training IsolationForest on 390755 normal samples, evaluating on 172519 samples...

✅ Training complete.
Predicted label counts:
pred_label
0    161678
1     10841
Name: count, dtype: int64
Mean anomaly score: -0.13385453171097553
              precision    recall  f1-score   support

           0       0.99      0.96      0.97    167468
           1       0.33      0.70      0.45      5051

    accuracy                           0.95    172519
   macro avg       0.66      0.83      0.71    172519
weighted avg       0.97      0.95      0.96    172519



In [25]:
import pandas as pd
import numpy as np
from pyod.models.iforest import IForest
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import (
    roc_auc_score,
    precision_recall_fscore_support
)

def tune_iforest_hyperparams(X_train, X_eval, y_train, y_eval,
                             param_grid=None,
                             score_metric='f1_anomaly',
                             n_jobs=-1, random_state=42):
    """
    Systematically explore IForest hyperparameters via combinatorial search,
    using only the anomaly (label==1) class metrics for evaluation.

    Parameters
    ----------
    X_train : array-like or sparse matrix
        Training feature matrix (normal samples only).
    X_eval : array-like or sparse matrix
        Evaluation feature matrix (mixed normal + anomalies).
    y_train, y_eval : array-like
        True labels for training and evaluation sets.
    param_grid : dict or None
        Dictionary of parameter lists to try (like sklearn's ParameterGrid).
        If None, defaults to a reasonable small search space.
    score_metric : {'f1_anomaly', 'recall_anomaly', 'precision_anomaly', 'roc_auc'}
        Metric used to rank models. 'roc_auc' still uses all labels.
    n_jobs : int
        Number of parallel threads for model training (-1 = all cores).
    random_state : int
        Random seed.

    Returns
    -------
    results_df : pd.DataFrame
        Each row corresponds to one parameter set and its evaluation metrics.
    best_model : IForest
        Trained model with best parameters.
    """

    if param_grid is None:
        param_grid = {
            'n_estimators': [2000],
            'max_samples': [0.5],
            'max_features': [0.5, 0.8, 1.0],
            'contamination': [0.01],
            'bootstrap': [False, True],
        }

    print(f"🔍 Starting grid search over {len(ParameterGrid(param_grid))} combinations...")

    X_train_dense = X_train.toarray()
    X_eval_dense = X_eval.toarray()

    records = []
    best_score = -np.inf
    best_model = None
    best_params = None

    for params in ParameterGrid(param_grid):
        print(f"\n▶ Training with params: {params}")

        model = IForest(
            **params,
            n_jobs=n_jobs,
            random_state=random_state
        )

        model.fit(X_train_dense)
        scores = model.decision_function(X_eval_dense)
        preds = model.predict(X_eval_dense)

        # Compute ROC-AUC across all samples
        try:
            roc_auc = roc_auc_score(y_eval, scores)
        except ValueError:
            roc_auc = np.nan

        # Compute precision, recall, f1 specifically for anomaly class
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_eval, preds, average=None, labels=[1]
        )
        precision_anom, recall_anom, f1_anom = precision[0], recall[0], f1[0]

        record = {
            **params,
            'roc_auc': roc_auc,
            'precision_anomaly': precision_anom,
            'recall_anomaly': recall_anom,
            'f1_anomaly': f1_anom
        }
        records.append(record)

        # Choose which metric to optimize
        metric_value = record.get(score_metric, np.nan)
        if np.isnan(metric_value):
            metric_value = -np.inf
        if metric_value > best_score:
            best_score = metric_value
            best_model = model
            best_params = params

        print(f"→ F1(anomaly): {f1_anom:.4f}, Recall(anomaly): {recall_anom:.4f}, "
              f"Precision(anomaly): {precision_anom:.4f}, ROC AUC: {roc_auc:.4f}")

    results_df = pd.DataFrame(records).sort_values(by=score_metric, ascending=False)
    print("\n✅ Best parameters found:")
    print(best_params)
    print(f"{score_metric.upper()}: {best_score:.4f}")

    return results_df, best_model

In [26]:
# Train only on normal samples
mask_normal = (y_train == 0)
X_train_normal = X_train[mask_normal]

results_df, best_model = tune_iforest_hyperparams(
    X_train=X_train_normal,
    X_eval=X_eval,
    y_train=y_train,
    y_eval=y_eval,
    score_metric='roc_auc',
    n_jobs=-1
)

# View ranked results
print(results_df.head(5))

# Use the best model to predict again if needed
y_pred = best_model.predict(X_eval.toarray())

🔍 Starting grid search over 6 combinations...

▶ Training with params: {'bootstrap': False, 'contamination': 0.01, 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 2000}
→ F1(anomaly): 0.8675, Recall(anomaly): 1.0000, Precision(anomaly): 0.7660, ROC AUC: 0.9982

▶ Training with params: {'bootstrap': False, 'contamination': 0.01, 'max_features': 0.8, 'max_samples': 0.5, 'n_estimators': 2000}
→ F1(anomaly): 0.8675, Recall(anomaly): 1.0000, Precision(anomaly): 0.7660, ROC AUC: 0.9984

▶ Training with params: {'bootstrap': False, 'contamination': 0.01, 'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 2000}
→ F1(anomaly): 0.8675, Recall(anomaly): 1.0000, Precision(anomaly): 0.7660, ROC AUC: 0.9984

▶ Training with params: {'bootstrap': True, 'contamination': 0.01, 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 2000}
→ F1(anomaly): 0.8649, Recall(anomaly): 1.0000, Precision(anomaly): 0.7620, ROC AUC: 0.9981

▶ Training with params: {'bootstrap': True, 'contamination

KeyboardInterrupt: 

In [27]:
import pandas as pd
import numpy as np
from pyod.models.ecod import ECOD
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support


def train_and_evaluate_ecod(X_train, X_eval, y_train, y_eval,
                            contamination=0.05):
    """
    Train and evaluate the ECOD anomaly detector (Empirical CDF-based Outlier Detection).

    Parameters
    ----------
    X_train : array-like or sparse matrix
        Training feature matrix (normal samples only).
    X_eval : array-like or sparse matrix
        Evaluation feature matrix (mixed normal + anomalies).
    y_train, y_eval : array-like
        Labels for training and evaluation sets.
    contamination : float
        Expected fraction of anomalies in the data.

    Returns
    -------
    result_df : pd.DataFrame
        Columns: ['true_label', 'pred_label', 'anomaly_score']
    model : ECOD
        Trained ECOD model.
    """

    print(f"Training ECOD on {X_train.shape[0]} normal samples, "
          f"evaluating on {X_eval.shape[0]} samples...")

    # Initialize and fit ECOD
    model = ECOD(contamination=contamination)
    model.fit(X_train.toarray())

    # Predict and score
    y_pred = model.predict(X_eval.toarray())          # 0 = normal, 1 = anomaly
    scores = model.decision_function(X_eval.toarray())  # higher = more normal

    # Compute evaluation metrics (anomaly-class focused)
    try:
        roc_auc = roc_auc_score(y_eval, scores)
    except ValueError:
        roc_auc = np.nan

    precision, recall, f1, _ = precision_recall_fscore_support(
        y_eval, y_pred, average=None, labels=[1]
    )
    precision_anom, recall_anom, f1_anom = precision[0], recall[0], f1[0]

    # Pack results
    result_df = pd.DataFrame({
        "true_label": y_eval,
        "pred_label": y_pred,
        "anomaly_score": scores
    })

    print("\n✅ Training complete.")
    print(f"F1(anomaly): {f1_anom:.4f}, Recall(anomaly): {recall_anom:.4f}, "
          f"Precision(anomaly): {precision_anom:.4f}, ROC AUC: {roc_auc:.4f}")

    return result_df, model

In [28]:

# Train and evaluate ECOD
result_df, ecod_model = train_and_evaluate_ecod(
    X_train=X_train_normal,
    X_eval=X_eval,
    y_train=y_train,
    y_eval=y_eval,
    contamination=0.05
)

# Evaluate performance
from sklearn.metrics import classification_report
print(classification_report(result_df["true_label"], result_df["pred_label"]))

Training ECOD on 390755 normal samples, evaluating on 172519 samples...

✅ Training complete.
F1(anomaly): 0.4462, Recall(anomaly): 0.7402, Precision(anomaly): 0.3193, ROC AUC: 0.9573
              precision    recall  f1-score   support

           0       0.99      0.95      0.97    167468
           1       0.32      0.74      0.45      5051

    accuracy                           0.95    172519
   macro avg       0.66      0.85      0.71    172519
weighted avg       0.97      0.95      0.96    172519

